In [8]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from google.cloud import storage # save the model to GCS

In [2]:
# Data stored in GCP, bucket name: 'gold-price-prediction' van TawabG
# gold_dataset = pd.read_csv("gs://gold-price-prediction/gold_dataset.csv") 
gold_dataset = pd.read_csv('gold_dataset.csv')
gold_dataset.head()

,date,timestamp,metal,exchange,currency,price,prev_close_price,ch,chp
0,2020-01-01T10:30:00.000Z,1.577875e+12,XAU,LBMA,EUR,1355.7059,1355.7059,0.0000,0.0000
1,2020-01-02T10:30:00.000Z,1.577961e+12,XAU,LBMA,EUR,1358.4830,1360.6718,-2.1889,-0.1611
2,2020-01-03T10:30:00.000Z,1.578047e+12,XAU,LBMA,EUR,1388.1762,1364.0890,24.0872,1.7352
3,2020-01-04T10:30:00.000Z,1.578134e+12,XAU,LBMA,EUR,1388.1762,1388.1762,0.0000,0.0000
4,2020-01-05T10:30:00.000Z,1.578220e+12,XAU,LBMA,EUR,1388.1762,1388.1762,0.0000,0.0000


In [3]:
def pre_processing(gold_dataset):
    
    gold_dataset = gold_dataset.dropna()
    gold_dataset['date'] = pd.to_datetime(gold_dataset['date'], errors='coerce')
    gold_dataset['date'] = gold_dataset['date'].dt.date
    gold_dataset = gold_dataset.set_index('date')
    
    return gold_dataset

In [4]:
preprocessed_gold_dataset = pre_processing(gold_dataset)
preprocessed_gold_dataset

,timestamp,metal,exchange,currency,price,prev_close_price,ch,chp
date,,,,,,,,
2020-01-01,1.577875e+12,XAU,LBMA,EUR,1355.7059,1355.7059,0.0000,0.0000
2020-01-02,1.577961e+12,XAU,LBMA,EUR,1358.4830,1360.6718,-2.1889,-0.1611
2020-01-03,1.578047e+12,XAU,LBMA,EUR,1388.1762,1364.0890,24.0872,1.7352
2020-01-04,1.578134e+12,XAU,LBMA,EUR,1388.1762,1388.1762,0.0000,0.0000
2020-01-05,1.578220e+12,XAU,LBMA,EUR,1388.1762,1388.1762,0.0000,0.0000
...,...,...,...,...,...,...,...,...
2020-11-15,1.605436e+12,XAU,LBMA,EUR,1589.6741,1589.6741,0.0000,0.0000
2020-11-16,1.605523e+12,XAU,LBMA,EUR,1599.8309,1587.6585,12.1724,0.7609
2020-11-17,1.605609e+12,XAU,LBMA,EUR,1586.7699,1592.8295,-6.0596,-0.3819


In [10]:
def create_features_dataframe(dataset):
    
    gold_features_df = pd.DataFrame()
    
    average_days_window_closing_price = [5, 30]
    for window in average_days_window_closing_price:
        gold_features_df['Mean__'+str(window)+'_days'] = dataset['price'].rolling(window).mean().shift(periods=1)
        gold_features_df['Std__'+str(window)+'_days'] = dataset['price'].rolling(window).std().shift(periods=1)
        gold_features_df['Max__'+str(window)+'_days'] = dataset['price'].rolling(window).max().shift(periods=1)
        gold_features_df['Min__'+str(window)+'_days'] = dataset['price'].rolling(window).min().shift(periods=1)
        gold_features_df['Min__'+str(window)+'_days'] = dataset['price'].rolling(window).sum().shift(periods=1)
        
    # get day of the week
    dataset['date'] = dataset.index
    dataset['day_of_week'] = dataset['date'].apply(lambda x: x.strftime("%A"))

    # get quarter
    dataset['month'] = dataset['date'].apply(lambda x: x.strftime("%B"))
    
    gold_features_df = pd.concat([gold_features_df, pd.get_dummies(dataset['day_of_week']), pd.get_dummies(dataset['month']), dataset['price'],  dataset['prev_close_price'],
                                     dataset['ch'], dataset['chp']],1)
    gold_features_df = gold_features_df.dropna()

    return gold_features_df

In [11]:
features_df = create_features_dataframe(preprocessed_gold_dataset)
features_df.tail()

,Mean__5_days,Std__5_days,Max__5_days,Min__5_days,Mean__30_days,Std__30_days,Max__30_days,Min__30_days,Friday,Monday,...,June,March,May,November,October,September,price,prev_close_price,ch,chp
date,,,,,,,,,,,,,,,,,,,,,
2020-11-15,1589.20476,3.737047,1594.5946,7946.0238,1614.277527,16.285805,1647.2692,48428.3258,0,0,...,0,0,0,1,0,0,1589.6741,1589.6741,0.0000,0.0000
2020-11-16,1589.57522,3.656623,1594.5946,7947.8761,1613.091833,16.748098,1647.2692,48392.7550,0,1,...,0,0,0,1,0,0,1599.8309,1587.6585,12.1724,0.7609
2020-11-17,1590.62248,5.656514,1599.8309,7953.1124,1612.244700,16.754918,1647.2692,48367.3410,0,0,...,0,0,0,1,0,0,1586.7699,1592.8295,-6.0596,-0.3819
2020-11-18,1591.12462,5.026801,1599.8309,7955.6231,1610.962200,17.192327,1647.2692,48328.8660,0,0,...,0,0,0,1,0,0,1581.7324,1588.6417,-6.9093,-0.4368
2020-11-19,1589.53628,6.606258,1599.8309,7947.6814,1609.663137,17.889077,1647.2692,48289.8941,0,0,...,0,0,0,1,0,0,1569.8107,1586.5450,-16.7343,-1.0660


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

X, y = features_df.drop('price', axis=1), features_df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

#create the model
clf = MLPRegressor()
clf.fit(X_train, y_train)

MLPRegressor()

In [19]:
X_train

,Mean__5_days,Std__5_days,Max__5_days,Min__5_days,Mean__30_days,Std__30_days,Max__30_days,Min__30_days,Friday,Monday,...,July,June,March,May,November,October,September,prev_close_price,ch,chp
date,,,,,,,,,,,,,,,,,,,,,
2020-05-21,1603.39786,13.739461,1621.9535,8016.9893,1576.138920,22.842776,1621.9535,47284.1676,0,0,...,0,0,0,1,0,0,0,1590.9545,-15.6818,-0.9955
2020-07-02,1577.52912,11.449625,1595.8929,7887.6456,1541.627120,25.619305,1595.8929,46248.8136,0,0,...,1,0,0,0,0,0,0,1583.7321,-13.7781,-0.8776
2020-10-21,1622.12438,4.855324,1625.2449,8110.6219,1616.248897,11.212408,1638.1607,48487.4669,0,0,...,0,0,0,0,0,1,0,1608.4627,10.6311,0.6566
2020-07-25,1606.38044,21.891014,1631.5041,8031.9022,1586.071110,16.110871,1631.5041,47582.1333,0,0,...,1,0,0,0,0,0,0,1631.5041,0.0000,0.0000
2020-02-05,1428.00786,3.625933,1430.3746,7140.0393,1410.729630,15.252395,1436.2358,42321.8889,0,0,...,0,0,0,0,0,0,0,1425.3833,-17.2367,-1.2241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-08,1618.04946,9.006071,1625.2344,8090.2473,1626.218043,19.061750,1655.4048,48786.5413,0,0,...,0,0,0,0,0,1,0,1604.7599,2.8474,0.1771
2020-08-10,1736.01610,12.654445,1744.5206,8680.0805,1639.316327,54.518559,1744.5206,49179.4898,0,1,...,0,0,0,0,0,0,0,1752.5291,-26.5238,-1.5367
2020-05-27,1584.82250,8.460020,1588.8206,7924.1125,1575.537810,20.833552,1621.9535,47266.1343,0,0,...,0,0,0,1,0,0,0,1567.4188,-16.0131,-1.0322


In [28]:
def calc_new_dates(gold_dataset):
    date_today = datetime.today().strftime('%Y-%m-%d')
    gold_dataset['date'] = pd.to_datetime(gold_dataset['date'], errors='coerce')
    gold_dataset['date'] = gold_dataset['date'].dt.date
    last_date=gold_dataset.iloc[-1]['date']
    date_today = datetime.strptime(date_today, '%Y-%m-%d')

    sdate = date_today   # start date
    edate = last_date   # end date

    list_dates = pd.date_range(edate,sdate-timedelta(days=0),freq='d').strftime("%Y%m%d").tolist()
    list_dates_int = list(map(int, list_dates))
    del list_dates_int[0]
    del list_dates_int[-1]
    return list_dates_int

In [29]:
calc_new_dates(gold_dataset)

[20201120,
 20201121,
 20201122,
 20201123,
 20201124,
 20201125,
 20201126,
 20201127,
 20201128,
 20201129,
 20201130]